In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm
import pickle

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook


#from dtale.views import startup
#import dtale
#import pyplot as plt
# os and glob help concat multiple csv files into one

Populating the interactive namespace from numpy and matplotlib
time: 1.59 s (started: 2021-12-20 18:15:02 -06:00)


In [3]:
AirBnBPermits = pd.read_csv('../data/Residential_Short_Term_Rental_Permits.csv')

time: 141 ms (started: 2021-12-20 18:15:56 -06:00)


In [4]:
AirBnBPermits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11876 entries, 0 to 11875
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Permit #                    11876 non-null  object 
 1   Applicant                   11873 non-null  object 
 2   Applicant Home Phone        75 non-null     object 
 3   Applicant Mobile Phone      6474 non-null   object 
 4   Applicant Work Phone        2026 non-null   object 
 5   Applicant Work Phone Ext.   2 non-null      object 
 6   Contact                     11473 non-null  object 
 7   Contact Home Phone          384 non-null    object 
 8   Contact Mobile Phone        6550 non-null   object 
 9   Contact Work Phone          4902 non-null   object 
 10  Contact Work Phone Ext.     2 non-null      object 
 11  Permit Subtype Description  11876 non-null  object 
 12  Permit Status               11876 non-null  object 
 13  Parcel                      118

In [5]:
#drops columns that have only NAN values
AirBnBPermits.dropna(how='all', axis=1, inplace=True)

time: 15 ms (started: 2021-12-20 18:16:52 -06:00)


In [6]:
# easy access to be able to drop columns (copy/paste)
list(AirBnBPermits.columns)

['Permit #',
 'Applicant',
 'Applicant Home Phone',
 'Applicant Mobile Phone',
 'Applicant Work Phone',
 'Applicant Work Phone Ext.',
 'Contact',
 'Contact Home Phone',
 'Contact Mobile Phone',
 'Contact Work Phone',
 'Contact Work Phone Ext.',
 'Permit Subtype Description',
 'Permit Status',
 'Parcel',
 'Date Entered',
 'Date Issued',
 'Expiration Date',
 'Address',
 'City',
 'State',
 'ZIP',
 'Subdivision / Lot',
 'Purpose',
 'Permit Owner Name',
 'Permit Owner Addr. 1',
 'Permit Owner Addr. 2',
 'Permit Owner Addr. 3',
 'Permit Owner City',
 'Permit Owner State',
 'Permit Owner ZIP',
 'Permit Type',
 'Permit Subtype',
 'Council District',
 'Census Tract',
 'Mapped Location']

time: 0 ns (started: 2021-12-20 18:21:10 -06:00)


In [10]:
#dropped all trash columns 
AirBnBPermits = AirBnBPermits.drop(['Applicant Home Phone', 'Applicant Mobile Phone', 'Applicant Work Phone', 'Applicant Work Phone Ext.', 'Contact', 'Contact Home Phone', 'Contact Mobile Phone', 'Contact Work Phone', 'Applicant Work Phone Ext.', 'Contact', 'Contact Home Phone','Contact Mobile Phone', 'Contact Work Phone', 'Contact Work Phone Ext.', 'Permit Owner Name',  'Permit Owner Addr. 1', 'Permit Owner Addr. 2', 'Permit Owner Addr. 3', 'Permit Owner City', 'Permit Owner State', 'Permit Owner ZIP', 'Census Tract'], axis=1)

time: 15 ms (started: 2021-12-20 18:26:03 -06:00)


In [15]:
AirBnBPermits.head()

,Permit #,Applicant,Permit Subtype Description,Permit Status,Parcel,Date Entered,Date Issued,Expiration Date,Address,City,State,ZIP,Subdivision / Lot,Purpose,Permit Type,Permit Subtype,Council District,Mapped Location
0,2016028278,john briggs,Short Term Rental - Non-Owner Occupied,EXPIRED,13011010400,06/13/2016 12:00:00 AM,2016-08-25,08/25/2020 12:00:00 AM,4517 BEACON DR,NASHVILLE,TN,37215,PT LOT 7 BLK S SEC 4 TYNE MEADE,By making this application for a Residential S...,CASR,CAZ10A002,34.0,"4517 BEACON DR\nNASHVILLE, TN 37215\n(36.08734..."
1,2018070986,"EVI Holdings, LLC",Short Term Rental � Multifamily,ISSUED,10404023000,11/08/2018 12:00:00 AM,2019-01-09,01/09/2022 12:00:00 AM,1208 16TH AVE S 6,NASHVILLE,TN,37212,PT LOT 24 HAYES ROKEBY PLAN,By making this application for Short Term Rent...,CASR,CAZ10A003,17.0,"1208 16TH AVE S 6\nNASHVILLE, TN 37212\n(36.14..."
2,2019036294,Kenneth Greenberg,Short Term Rental � Multifamily,CANCELLED,072050F00300CO,06/19/2019 12:00:00 AM,2019-06-26,06/26/2021 12:00:00 AM,1027 E TRINITY LN 3,NASHVILLE,TN,37216,UNIT 3 TRINITY LANE TOWNHOMES,By making this application for a Residential S...,CASR,CAZ10A003,5.0,"1027 E TRINITY LN 3\nNASHVILLE, TN 37216\n(36...."
3,2018074155,"Niido Nashville, LLC",Short Term Rental � Multifamily,ISSUED,09310023500,11/27/2018 12:00:00 AM,2018-12-26,12/26/2022 12:00:00 AM,501 REP JOHN LEWIS WAY S 507,NASHVILLE,TN,37203,LOTS 20-25 & PT LOT 19 BARROW GROVE,By making this application for Short Term Rent...,CASR,CAZ10A003,19.0,"501 REP JOHN LEWIS WAY S 507\nNASHVILLE, TN 37203"
4,201514398,"Smallman, William",Short Term Rental - Non-Owner Occupied,EXPIRED,10511016200,04/16/2015 12:00:00 AM,2015-04-16,04/16/2021 12:00:00 AM,1801 NEAL TER,NASHVILLE,TN,37203,LOT 9 BLK A S M NEAL,By making this application for a Residential S...,CASR,CAZ10A002,17.0,"1801 NEAL TER\nNASHVILLE, TN 37203\n(36.13453,..."


time: 15 ms (started: 2021-12-20 18:43:23 -06:00)


In [13]:
#turn to datetime to isolate only year 2021
AirBnBPermits['Date Issued'] = pd.to_datetime(AirBnBPermits['Date Issued'])

time: 203 ms (started: 2021-12-20 18:32:22 -06:00)


In [14]:
AirBnBPermits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11876 entries, 0 to 11875
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Permit #                    11876 non-null  object        
 1   Applicant                   11873 non-null  object        
 2   Permit Subtype Description  11876 non-null  object        
 3   Permit Status               11876 non-null  object        
 4   Parcel                      11876 non-null  object        
 5   Date Entered                11876 non-null  object        
 6   Date Issued                 10541 non-null  datetime64[ns]
 7   Expiration Date             11872 non-null  object        
 8   Address                     11876 non-null  object        
 9   City                        11875 non-null  object        
 10  State                       11875 non-null  object        
 11  ZIP                         11876 non-null  int64     

In [16]:
#example for range in dates
#https://www.interviewqs.com/ddi-code-snippets/select-pandas-dataframe-rows-between-two-dates

start_date = '01-01-2021' 
end_date = '12-30-2021'
mask = (AirBnBPermits['Date Issued'] > start_date) & (AirBnBPermits['Date Issued'] <= end_date)

time: 0 ns (started: 2021-12-20 18:48:50 -06:00)


In [17]:
#new df
AirBnBPermits2021 = AirBnBPermits.loc[mask]

time: 0 ns (started: 2021-12-20 18:48:51 -06:00)


In [18]:
AirBnBPermits2021.head()

,Permit #,Applicant,Permit Subtype Description,Permit Status,Parcel,Date Entered,Date Issued,Expiration Date,Address,City,State,ZIP,Subdivision / Lot,Purpose,Permit Type,Permit Subtype,Council District,Mapped Location
51,2021008680,"KAMEL, RAAFAT",Short Term Rental � Multifamily,ISSUED,071120E04700CO,02/09/2021 12:00:00 AM,2021-02-19,02/19/2022 12:00:00 AM,1107 HARMONY WAY,NASHVILLE,TN,37207,UNIT 47 LYRIC AT CLEVELAND PARK,By making this application for a Residential S...,CASR,CAZ10A003,5.0,"1107 HARMONY WAY\nNASHVILLE, TN 37207"
54,2021015610,CITY HEIGHTS VIEWS LLC,Short Term Rental � Multifamily,ISSUED,092100N01100CO,03/16/2021 12:00:00 AM,2021-03-26,03/26/2022 12:00:00 AM,2910 FELICIA ST 11,NASHVILLE,TN,37209,UNIT 11 RUDY AT CITY HEIGHTS,By making this application for a Residential S...,CASR,CAZ10A003,21.0,"2910 FELICIA ST 11\nNASHVILLE, TN 37209\n(36.1..."
69,2020065139,"BEUTLER TN INVESTMENTS, LLC",Short Term Rental � Multifamily,ISSUED,092070P00100CO,10/16/2020 12:00:00 AM,2021-02-19,02/19/2022 12:00:00 AM,901 21ST AVE N,NASHVILLE,TN,37208,UNIT 901 RESIDENCES AT 21ST AND HERMAN,By making this application for a Residential S...,CASR,CAZ10A003,21.0,"901 21ST AVE N\nNASHVILLE, TN 37208\n(36.16291..."
72,2021068672,Grand Welcome Nashville,Short Term Rental � Multifamily,ISSUED,072091C00500CO,10/26/2021 12:00:00 AM,2021-11-02,11/02/2022 12:00:00 AM,844 CHEROKEE AVE 5,NASHVILLE,TN,37207,UNIT 5 TOWNHOMES AT 844,By making this application for a Residential S...,CASR,CAZ10A003,5.0,"844 CHEROKEE AVE 5\nNASHVILLE, TN 37207\n(36.2..."
75,2020065161,"BEUTLER TN INVESTMENTS, LLC",Short Term Rental � Multifamily,ISSUED,092070P00300CO,10/16/2020 12:00:00 AM,2021-02-19,02/19/2022 12:00:00 AM,905 21ST AVE N,NASHVILLE,TN,37208,UNIT 905 RESIDENCES AT 21ST AND HERMAN,By making this application for a Residential S...,CASR,CAZ10A003,21.0,"905 21ST AVE N\nNASHVILLE, TN 37208\n(36.16295..."


time: 16 ms (started: 2021-12-20 18:49:00 -06:00)


In [19]:
AirBnBPermits2021['Date Issued'].value_counts()

2021-02-25    44
2021-09-27    32
2021-12-03    26
2021-10-11    23
2021-12-14    19
              ..
2021-02-02     1
2021-02-10     1
2021-10-27     1
2021-02-11     1
2021-01-27     1
Name: Date Issued, Length: 230, dtype: int64

time: 0 ns (started: 2021-12-20 18:50:31 -06:00)


In [20]:
#37203 is the center of nashville (gulch/partheon/music row)
AirBnBPermits2021['ZIP'].value_counts()

37203    223
37207    196
37208    157
37212    152
37206    134
37209    108
37210     52
37201     42
37211     32
37216     31
37204     29
37214     24
37115     19
37217     16
37076      9
37218      8
37219      8
37205      4
37215      4
37138      3
37189      2
37072      2
37013      2
37027      1
37080      1
37220      1
37221      1
Name: ZIP, dtype: int64

time: 15 ms (started: 2021-12-20 18:52:50 -06:00)


In [21]:
AirBnBPermits2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1261 entries, 51 to 11852
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Permit #                    1261 non-null   object        
 1   Applicant                   1261 non-null   object        
 2   Permit Subtype Description  1261 non-null   object        
 3   Permit Status               1261 non-null   object        
 4   Parcel                      1261 non-null   object        
 5   Date Entered                1261 non-null   object        
 6   Date Issued                 1261 non-null   datetime64[ns]
 7   Expiration Date             1261 non-null   object        
 8   Address                     1261 non-null   object        
 9   City                        1261 non-null   object        
 10  State                       1261 non-null   object        
 11  ZIP                         1261 non-null   int64     

In [23]:
AirBnBPermits2021 = AirBnBPermits2021.rename(columns={'Address': 'Street'})

time: 0 ns (started: 2021-12-20 19:30:37 -06:00)


In [24]:
AirBnBPermits2021.to_csv("AirBnBPermits2021.csv", sep=',', index=False, header=True)

time: 47 ms (started: 2021-12-20 19:30:38 -06:00)
